In [1]:
import os
os.chdir("..")

In [2]:
from fix_data import add_label_T
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import xgboost as xgb
import lightgbm as lgb
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

In [3]:
df = pd.read_csv("data/DropSeq/HCC1806_Filtered_Normalised_3000_Data_train.txt", sep=" ")
df = add_label_T(df, dropname=True)
df

,H1-5,MALAT1,MT-RNR2,ARVCF,BCYRN1,ATXN7L2,IGFBP3,H1-3,CTIF,RNF123,...,CDKN3,DLD,PMPCA,ZNF165,SCCPDH,NTAN1,CLIP2,DUSP23,ZNF682,label
0,2,3,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2,3,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,5,2,0,0,1,0,1,3,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,12,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14677,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14678,0,4,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14679,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14680,3,3,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df = df.sample(frac=1)

In [5]:
X = df.drop("label", axis=1)
y = df["label"]

In [6]:
def get_batch(n, batch_size):
    start_idx = n * batch_size
    end_idx = (n + 1) * batch_size
    X_batch = X[start_idx:end_idx]
    y_batch = y[start_idx:end_idx]

    return X_batch, y_batch

In [7]:
batch_size = 2447
num_batches = 6

In [8]:
X_train_val, X_test, y_train_val, y_val = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25)

In [11]:
clf = LogisticRegression(penalty="l1", solver="liblinear")
param_grid = {
    "C": np.arange(0.5, 1.5, 0.1)
}
grid_search = GridSearchCV(clf, param_grid)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_, grid_search.best_score_)

{'C': 0.6} 0.9469803234541306


In [12]:
clf = LogisticRegression(penalty="l2", solver="liblinear")
param_grid = {
    "C": [10**x for x in range(-3, 3)]
}
grid_search = GridSearchCV(clf, param_grid)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_, grid_search.best_score_)

{'C': 0.1} 0.9491370280919481


In [16]:
clf = LogisticRegression(penalty="elasticnet", solver="saga", C=1, l1_ratio=0.5, max_iter=1000)
clf.fit(X_train, y_train)
clf.score(X_val, y_val)

In [ ]:
clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)
clf.score(X_val, y_val)

0.9482465100442629

In [ ]:
clf = xgb.XGBClassifier()
clf.fit(X_train, y_train)
clf.score(X_val, y_val)

0.9540347293156282

In [ ]:
clf = LogisticRegression(penalty="l2", C=0.1, solver="liblinear")
clf.fit(X_train, y_train)
clf.score(X_val, y_val)

0.9588014981273408